# Instructions for Loading Data

Adapted from: https://github.com/pillowlab/psytrack_learning/blob/main/PsyTrack_Learning_Examples.ipynb

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import re
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy.special import expit
import numpy as np
import pandas as pd

import psytrack_learning as psy
from psytrack_learning.getMAP import getMAP
from psytrack_learning.helper.helperFunctions import update_hyper, hyper_to_list
from psytrack_learning.helper.jacHessCheck import compHess, compHess_nolog
from psytrack_learning.helper.invBlkTriDiag import getCredibleInterval
from psytrack_learning.hyperparameter_optimization import evd_lossfun
from psytrack_learning.learning_rules import RewardMax, PredictMax, REINFORCE, REINFORCE_base
from psytrack_learning.simulate_learning import reward_max, predict_max, reinforce, reinforce_base 
from psytrack_learning.simulate_learning import simulate_learning


# Set matplotlib defaults from making files editable and consistent in Illustrator
colors = psy.COLORS
zorder = psy.ZORDER
plt.rcParams['figure.dpi'] = 140
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.facecolor'] = (1,1,1,0)
plt.rcParams['savefig.bbox'] = "tight"
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'cmu serif'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['axes.labelsize'] = 12

# Set save path for preprocessed data and all figures
spath = "./Figures/" # You can change this 
sim_colors = ["#D81159", "#4357AD", "#EE8434", "#CC3399", "#409091"]

## Download and pre-process IBL mouse data

1) Download the [IBL dataset](https://doi.org/10.6084/m9.figshare.11636748.v7) (version 7, uploaded Feb 7, 2020).

2) Update the `ibl_data_path` variable below to where the `ibl-behavioral-data-Dec2019` directory exists on your computer.

In [ ]:
ibl_data_path = "./Figures/" # You can change this 

3) We will also need to install the [ONE Light](https://github.com/int-brain-lab/ibllib/tree/master/oneibl) Python library (from the IBL) with `pip install ibllib`. This allows us to build a table of all the subject and session data contained within the dataset.

In [ ]:
import os
from oneibl.onelight import ONE
import pandas as pd

current_cwd = os.getcwd()
os.chdir(ibl_data_path)

# Search all sessions that have these dataset types.
required_vars = ['_ibl_trials.choice', '_ibl_trials.contrastLeft',
                 '_ibl_trials.contrastRight','_ibl_trials.feedbackType']
one = ONE()
eids = one.search(required_vars)

mouseData = pd.DataFrame()
for eid in eids:
    lab, _, subject, date, session = eid.split("/")    
    sess_vars = {
        "eid": eid,
        "lab": lab,
        "subject": subject,
        "date": date,
        "session": session,
    }
    mouseData = mouseData.append(sess_vars, sort=True, ignore_index=True)

os.chdir(current_cwd)

4) Next, we will use the table of session data to process the raw trial data below into a single CSV file, `ibl_processed.csv`, saved locally.

There are several known anomalies in the raw data:
 - CSHL_002 codes left contrasts as negative right contrasts on 81 trials (these trials are corrected)
 - ZM_1084 has `feedbackType` of 0 for 3 trials (these trials are omitted)
 - DY_009, DY_010, DY_011 each have <5000 trials total (no adjustment)
 - ZM_1367, ZM_1369, ZM_1371, ZM_1372, and ZM_1743 are shown non-standard contrast values of 0.04 and 0.08 (no adjustment)

_2 min_

In [ ]:
all_vars = ["contrastLeft", "contrastRight", "choice", "feedbackType", "probabilityLeft"]
df = pd.DataFrame()

all_mice = []
for j, s in enumerate(mouseData["subject"].unique()):
    print("\rProcessing " + str(j+1) + " of " + str(len(mouseData["subject"].unique())), end="")
    mouse = mouseData[mouseData["subject"]==s].sort_values(['date', 'session']).reset_index()
    for i, row in mouse.iterrows():
        myVars = {}
        for v in all_vars:
            filename = "_ibl_trials." + v + ".npy"
            var_file = os.path.join(ibl_data_path, row.eid, "alf", filename)
            myVars[v] = list(np.load(var_file).flatten())

        num_trials = len(myVars[v])
        myVars['lab'] = [row.lab]*num_trials
        myVars['subject'] = [row.subject]*num_trials
        myVars['date'] = [row.date]*num_trials
        myVars['session'] = [row.session]*num_trials

        all_mice += [pd.DataFrame(myVars, columns=myVars.keys())]
        
df = pd.concat(all_mice, ignore_index=True)

df = df[df['choice'] != 0]        # dump mistrials
df = df[df['feedbackType'] != 0]  # 3 anomalous trials from ZM_1084, omit
df.loc[np.isnan(df['contrastLeft']), "contrastLeft"] = 0
df.loc[np.isnan(df['contrastRight']), "contrastRight"] = 0
df.loc[df["contrastRight"] < 0, "contrastLeft"] = np.abs(df.loc[df["contrastRight"] < 0, "contrastRight"])
df.loc[df["contrastRight"] < 0, "contrastRight"] = 0  # 81 anomalous trials in CSHL_002, correct
df["answer"] = df["feedbackType"] * df["choice"]      # new column to indicate correct answer
df.loc[df["answer"]==1, "answer"] = 0
df.loc[df["answer"]==-1, "answer"] = 1
df.loc[df["feedbackType"]==-1, "feedbackType"] = 0
df.loc[df["choice"]==1, "choice"] = 0
df.loc[df["choice"]==-1, "choice"] = 1
df.to_csv(spath+"ibl_processed.csv", index=False)

5) Next we do a few sanity checks on our data, to make sure everything processed correctly.

In [ ]:
print("contrastLeft: ", np.unique(df['contrastLeft']))   # [0, 0.0625, 0.125, 0.25, 0.5, 1.0] and [0.04, 0.08]
print("contrastRight: ", np.unique(df['contrastRight'])) # [0, 0.0625, 0.125, 0.25, 0.5, 1.0] and [0.04, 0.08]
print("choice: ", np.unique(df['choice']))               # [0, 1]
print("feedbackType: ", np.unique(df['feedbackType']))   # [0, 1]
print("answer: ", np.unique(df['answer']))               # [0, 1]

6) Finally, we define a function `getMouse()` that extracts the data for a single mouse from our CSV file, and returns it in a PsyTrack compatible dictionary. We will use this function to access IBL mouse data in the figures below. Note the keyword argument and default value $p=5$ which controls the strength of the $\tanh$ transformation on the contrast values. See Figure S3 and the STAR Methods for more details.

**Note:** Once steps 1-6 have been run once, only step 6 will need to be run on subsequent uses.

In [3]:
mouse_data_path = spath + "ibl_processed.csv"   # --- UPDATE if necessary ---
MOUSE_DF = pd.read_csv(mouse_data_path)

def getMouse(subject, p=5):
    df = MOUSE_DF[MOUSE_DF['subject']==subject]   # Restrict data to the subject specified
    
    cL = np.tanh(p*df['contrastLeft'])/np.tanh(p)   # tanh transformation of left contrasts
    cR = np.tanh(p*df['contrastRight'])/np.tanh(p)  # tanh transformation of right contrasts
    cBoth = cR - cL
    inputs = dict(cL = np.array(cL)[:, None], cR = np.array(cR)[:, None], cBoth = np.array(cBoth)[:, None])

    dat = dict(
        subject=subject,
        lab=np.unique(df["lab"])[0],
        contrastLeft=np.array(df['contrastLeft']),
        contrastRight=np.array(df['contrastRight']),
        date=np.array(df['date']),
        dayLength=np.array(df.groupby(['date','session']).size()),
        correct=np.array(df['feedbackType']),
        answer=np.array(df['answer']),
        probL=np.array(df['probabilityLeft']),
        inputs = inputs,
        y = np.array(df['choice'])
    )
    
    return dat

---